In [3]:
from importlib import reload
import load_data
reload(load_data)
from load_data import *


import numpy as np
from langdetect import detect
import translitcodec
import codecs

# 1. Standardowa analiza

In [5]:
path_to_file = "data/title.akas.tsv.gz"
df1 = load_to_dataframe(path_to_file)
df1 = df1.drop(columns=["language", "types", "attributes"])

path_to_file = "data/title.basics.tsv.gz"
df3 = load_to_dataframe(path_to_file)
df3 = df3.loc[:, ["tconst", "titleType"]]

path_to_file = "data/title.ratings.tsv.gz"
df7 = load_to_dataframe(path_to_file)

df = df1.merge(df3,
              how="left",
              left_on="titleId",
              right_on="tconst"
)

# wybieramy tylko "movie" i niektóre kolumny
df = df.loc[df["titleType"] == "movie", ["titleId", "title", "region", "isOriginalTitle"]]

# zamieniamy wszystkie tytuły na lowercase i dekodujemy narodowe znaki
df["title"] = df["title"].astype(str).str.lower().apply(lambda x: codecs.encode(str(x), 'translit/short'))

# wydostajemy kandydatów na filmy z których odzyskamy kraje
# zgodnie z uwagą w PDFie od prowadzących
df_think = df.loc[df["isOriginalTitle"] == 0].merge(
    df.loc[df["isOriginalTitle"] == 1],
    on="titleId",
    how="left",
    suffixes=("", "_y")
).query(
    "title == title_y"
).drop(
    labels=["title_y", "region_y", "isOriginalTitle", "isOriginalTitle_y"],
    axis=1
)

# grupujemy po ID tytułu
# potem zliczamy ile było przypisanych regionów
# tam, gdzie był jeden, daje się zidentyfikować jednoznacznie
disambig = df_think.loc[:, ["titleId", "region"]].groupby(
    by=["titleId"]
).count()["region"] == 1

df_with_known_countries = df_think.loc[df_think["titleId"].isin(disambig.index[disambig == True])]

df_analysis = df_with_known_countries.merge(df7,
                                            how="left",
                                            left_on="titleId",
                                            right_on="tconst"
).drop(
    columns=["tconst"]
).dropna()

c:\Users\priva\OneDrive\Desktop\STUDIA\Narzędzia Informatyczne w Pythonie wspierające Analizę Danych\Projekt zaliczeniowy\load_data.py:25: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(path_to_file,


# 2. Model językowy

In [6]:
# wspieramy rozpoznawanie języka przez bibliotekę 'langdetect'
def mydetect(x):
    try:
        return detect(x).upper()
    except:
        return pd.NA

# funkcje do .apply()
def myfunc_EC(x):
    if x[5] != 'ES':
        x[2] = x[5]
    return x

def myfunc_TR(x):
    if x[5] != 'TR':
        x[2] = x[5]
    return x
    
# usuwamy dziwne filmy z Ekwadoru
df_EC = df_analysis.loc[df_analysis["region"] == 'EC'].copy()
df_EC.loc[:, "detected"] = df_EC["title"].apply(mydetect)

false_EC = df_EC.loc[df_EC["detected"] != 'ES']["titleId"]
df_EC = df_EC.apply(myfunc_EC, axis=1).drop(columns=["detected"])

df_analysis.loc[df_EC.index] = df_EC

In [8]:
# # usuwamy dziwne filmy z Turcji
# df_TR = df_analysis.loc[df_analysis["region"] == 'TR'].copy()
# df_TR.loc[:, "detected"] = df_TR["title"].apply(mydetect)
# df_TR = df_TR.dropna()

# false_TR = df_TR.loc[df_TR["detected"] != 'TR']["titleId"]
# df_TR = df_TR.apply(myfunc_TR, axis=1).drop(columns=["detected"])

# df_analysis.loc[df_TR.index] = df_TR

# 3. Zasadnicza część zadania

In [ ]:
K = 10000 # próg odcięcia
N = 1000 # ile pierwszych filmów bierzemy
df_analysis.loc[df_analysis["numVotes"] > K].sort_values(
    by="averageRating", ascending=False
).head(N).loc[:, ["titleId", "region"]].groupby(by="region").count()

In [98]:
from importlib import reload
import task1
reload(task1)
from task1 import *

In [11]:
df, df3 = load_dataframes(path1="data/title.akas.tsv.gz",
                                path2="data/title.basics.tsv.gz",
                                path3="data/title.ratings.tsv.gz")

c:\Users\priva\OneDrive\Desktop\STUDIA\Narzędzia Informatyczne w Pythonie wspierające Analizę Danych\Projekt zaliczeniowy\load_data.py:25: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(path_to_file,


In [99]:
df_analysis = prepareDataset(df, df3)

In [ ]:
rank = answerRanks(df_analysis, Ks = [10, 100, 1000], Ns = range(10, 201, 10), file = "answer.txt")

In [73]:
rank = {}
for K in [10, 100, 1000]:
    print(f"Threshold: {K}")
    rank[K] = {}
    for N in range(10, 201, 10):
        pa = properAnalysis(df_analysis, K = K, N = N)
        rank[K][N] = pa[:min(10, len(pa))]
        print()
    print("-"*40)

Threshold: 10
Top 10 countries - ranking
1. United States
2. Spain
3. India
4. Norway
5. El Salvador
6. Thailand

Top 20 countries - ranking
1. United States
2. India
3. Spain
4. Brazil
5. Italy
6. Norway
7. El Salvador
8. Thailand

Top 30 countries - ranking
1. United States
2. India
3. Brazil
4. Spain
5. Thailand
6. United Kingdom
7. Iran, Islamic Republic of
8. Italy
9. Norway
10. El Salvador

Top 40 countries - ranking
1. United States
2. India
3. Thailand
4. Brazil
5. Spain
6. United Kingdom
7. Canada
8. Colombia
9. Iran, Islamic Republic of
10. Italy

Top 50 countries - ranking
1. United States
2. India
3. Thailand
4. Brazil
5. Spain
6. United Kingdom
7. Italy
8. Canada
9. Colombia
10. Germany

Top 60 countries - ranking
1. United States
2. India
3. Spain
4. Italy
5. Thailand
6. Brazil
7. United Kingdom
8. Canada
9. Colombia
10. Germany

Top 70 countries - ranking
1. United States
2. India
3. Spain
4. Italy
5. Thailand
6. Brazil
7. Canada
8. United Kingdom
9. Iran, Islamic Republ